<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Pr%C3%A9diction_conforme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Projet : Classification et  Prédiction conforme**
Le jeu de données **CIFAR-10** contient 60 000 images en couleur de taille 32x32. Ces images représentent des objets appartenant à 10 classes :

*   avion
*   automobile
*   oiseau
*   chat
*   cerf
*   chien
*   grenouille
*   cheval
*   bateau
*   camion

Le jeu de données d'entraînement contient 50 000 images d'entraînement, soit 5 000 images par classe. Le jeu de test contient 10 000 images de test, soit 1 000 images par classe.

**Pourquoi ce jeu de donnée ?**

CIFAR-10 comme jeu de données est un bon point de départ pour entraîner et tester des Convolutional Neural Networks (CNN), car il n'est pas trop volumineux mais suffisamment varié pour évaluer la performance d'un modèle.

**Importation des données et statistiques simples**

Tu dois dire pourquoi tu choisis ce jeu de donnée. Tu dois également faire de la regression linéaire sur ce jeu de donnée et dire si ça marche ou pas et pourquoi. Vu que tu as une CNN tu peux peut être penser faire de la CV+,  pour faire de la prédiction conforme. Il semble que la SCP serait judicieux

In [ ]:
from sklearn.model_selection import train_test_split

# Charger les données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# Diviser les données en trois ensembles : entraînement, calibration et test
X_train, X_calibration, y_train, y_calibration = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# Normalisation des données
X_train = X_train.astype('float32') / 255.0
X_calibration = X_calibration.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Entraîner le modèle sur l'ensemble d'entraînement
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Prédire sur l'ensemble de calibration
y_pred_calibration = model.predict(X_calibration)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 46s 79ms/step - accuracy: 0.3278 - loss: 1.8394 - val_accuracy: 0.4893 - val_loss: 1.4549
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 82s 80ms/step - accuracy: 0.5255 - loss: 1.3333 - val_accuracy: 0.5548 - val_loss: 1.2339
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 81s 77ms/step - accuracy: 0.5892 - loss: 1.1666 - val_accuracy: 0.6060 - val_loss: 1.1226
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 43s 77ms/step - accuracy: 0.6307 - loss: 1.0496 - val_accuracy: 0.6245 - val_loss: 1.0843
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 44s 77ms/step - accuracy: 0.6660 - loss: 0.9694 - val_accuracy: 0.6410 - val_loss: 1.0215
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 86s 85ms/step - accuracy: 0.6900 - loss: 0.8993 - val_accuracy: 0.6570 - val_loss: 0.9817
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 83s 86ms/step - accuracy: 0.7078 - loss: 0.8461 - val_accuracy: 0.6545 - val_loss: 0.9899
Epoch 8/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 77s 78ms/step - accuracy: 0.7188 - loss: 0.8027 - 

# **La prédiction conforme**

On fera une SCP parceque nous avons beaucoup de données.

In [ ]:
import numpy as np
# Calculer les scores de conformité sur l'ensemble de calibration
def calculer_scores_conformite(y_pred, y_true):
    scores_conformite = []
    for i in range(len(y_true)):
        prob_vraie_classe = y_pred[i][y_true[i][0]]
        score = 1 - prob_vraie_classe
        scores_conformite.append(score)
    return np.array(scores_conformite)

scores_conformite_calibration = calculer_scores_conformite(y_pred_calibration, y_calibration)

# Définir un seuil basé sur les scores de calibration et le niveau de confiance
alpha = 0.05
seuil = np.percentile(scores_conformite_calibration, 100 * (1 - alpha))

# Appliquer ce seuil à l'ensemble de test
y_pred_test = model.predict(X_test)
pred_intervals = [np.where(p >= 1 - seuil)[0] for p in y_pred_test]

# Afficher quelques résultats de l'ensemble de test
for i in range(5):
    print(f"Image {i+1}: Intervalles de prédiction {pred_intervals[i]}, Classe réelle: {y_test[i][0]}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step
Image 1: Intervalles de prédiction [0 3 5], Classe réelle: 3
Image 2: Intervalles de prédiction [0 1 8], Classe réelle: 8
Image 3: Intervalles de prédiction [0 1 8 9], Classe réelle: 8
Image 4: Intervalles de prédiction [0 8], Classe réelle: 0
Image 5: Intervalles de prédiction [2 3 4 6], Classe réelle: 6
